In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import wfdb
import csv
import os
from scipy.signal import butter, filtfilt
#from PyEMD import EMD
import emd
import time
import pywt
import biosppy.signals.ecg as bsp_ecg
import biosppy.signals.tools as bsp_tools
import torch
import torch.nn as nn
from scipy.signal import find_peaks
from scipy.io import loadmat
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import scipy.signal as signal


from utils import print_signal_qrs, print_signal, calcul_f1, perf

In [10]:
df = pd.read_csv('data_csv/mit_bih_Arrhythmia/207.csv') #207
mini = 0
maxi = 10000

ecg_signal = np.array(df["MLII"], dtype=np.float32)[mini:maxi]
fs = 360
QRS = df["labels"].dropna().tolist()
QRS = [int(R) for R in QRS if R <= maxi and R <= maxi]

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        encoded = F.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        return decoded, encoded

class StackedAutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(StackedAutoEncoder, self).__init__()
        self.autoencoders = nn.ModuleList()
        prev_dim = input_dim
        
        for h_dim in hidden_dims:
            self.autoencoders.append(AutoEncoder(prev_dim, h_dim))
            prev_dim = h_dim
        
        self.classifier = nn.Linear(hidden_dims[-1], output_dim)
    
    def forward(self, x):
        for ae in self.autoencoders:
            x, _ = ae(x)
        x = F.relu(x)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

def train_autoencoder(ae, data, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(ae.parameters(), lr=lr)
    for epoch in range(epochs):
        for batch in data:
            inputs, _ = batch
            optimizer.zero_grad()
            outputs, _ = ae(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Fonction d'entraînement du classificateur final
def train_classifier(model, data, epochs=50, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for batch in data:
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [12]:
def preprocess_ecg(ecg_signal, sampling_rate):
    # Filtrage passe-bas (cutoff = 50 Hz)
    low_pass_cutoff = 50.0
    b, a = signal.butter(4, low_pass_cutoff / (0.5 * sampling_rate), btype='low')
    ecg_low = signal.filtfilt(b, a, ecg_signal)
    
    # Filtrage passe-haut (cutoff = 0.5 Hz)
    high_pass_cutoff = 0.5
    b, a = signal.butter(4, high_pass_cutoff / (0.5 * sampling_rate), btype='high')
    ecg_band = signal.filtfilt(b, a, ecg_low)
    
    # Filtrage notch (notch_freq = 50 Hz)
    notch_freq = 50.0
    b, a = signal.iirnotch(notch_freq, Q=30, fs=sampling_rate)
    ecg_filtered = signal.filtfilt(b, a, ecg_band)
    
    # Normalisation
    ecg_normalized = (ecg_filtered - np.mean(ecg_filtered)) / np.std(ecg_filtered)
    
    return ecg_normalized

def segment_ecg(ecg_signal, segment_length, overlap=0.5):
    segments = []
    step = int(segment_length * (1 - overlap))
    for start in range(0, len(ecg_signal) - segment_length + 1, step):
        segments.append(ecg_signal[start:start + segment_length])
    return np.array(segments)

# Pré-traitement du signal ECG
ecg_preprocessed = preprocess_ecg(ecg_signal, fs)

scaler = MinMaxScaler()
ecg_signals_normalized = scaler.fit_transform(ecg_signals)

segment_length = 300
segments = [ecg_signals_normalized[i:i+segment_length] for i in range(0, len(ecg_signals_normalized), segment_length)]

# Conversion en format adapté pour le modèle
train_data = np.array(segments)


input_dim = 300  # dimension d'entrée des signaux ECG, supposons 300 échantillons par signal
hidden_dims = [75, 15]  # dimensions des couches cachées de l'auto-encodeur
output_dim = 2  # nombre de classes pour la classification binaire (présence ou absence de QRS)

# Initialiser le modèle
model = StackedAutoEncoder(input_dim, hidden_dims, output_dim)

# Pré-entraîner chaque auto-encodeur
for ae in model.autoencoders:
    train_autoencoder(ae, train_data, epochs=50)

# Affiner l'ensemble du modèle
train_classifier(model, train_data, epochs=50)



# Transmettre les segments au modèle
input_data = torch.tensor(ecg_segments, dtype=torch.float32)

# Utilisation du modèle créé précédemment
output = model(input_data)

NameError: name 'model' is not defined

In [4]:
r_peaks = find_peaks(ecg_signal)[0].tolist()

In [5]:
print_signal_qrs(ecg_signal, "predicted", r_peaks)

TypeError: '<' not supported between instances of 'str' and 'int'

In [5]:
#Error = 0.5 * [(y[i] - t[i]) **2 for i in range(len(t))]

L'algo :

Détection peaks primaire : ?

Selection des QRS: Stacked-auto-encoder pour choisir les QRS intéressant, fênetre de 54 autour du peak (26 avant, 27 après)

Affinage de la selection : MLP avec 20 neurones +  les RR intervals pour estimer mieux les QRS

QRS selection final : MLP avec 4 neurones

In [12]:
def scaled_conjugate_gradient(f, x0, args=(), **kwargs):
    # ... Implémentation de l'algorithme ...
    return x

class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, output_size),
            nn.ReLU(True)  # ou nn.Sigmoid() si les valeurs de sortie doivent être entre 0 et 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

sizes = [(54, 30, 54), (54, 20, 54), (54, 10, 1)]
autoencoders = [Autoencoder(*size) for size in sizes]
criterion = nn.MSELoss()
optimizers = [optim.Adam(ae.parameters(), lr=0.001) for ae in autoencoders]

x = torch.randn(10, 54)  # 10 exemples, chaque exemple de taille 54

for ae, optimizer in zip(autoencoders, optimizers):
    for epoch in range(100):  # Nombre d'époques de formation
        optimizer.zero_grad()
        y_pred = ae(x)
        loss = criterion(y_pred, x)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

def stacked_autoencoders(autoencoders, x):
    for ae in autoencoders:
        x = ae(x)
    return x

y = stacked_autoencoders(autoencoders, x)
print(y)

Epoch 10, Loss: 1.142287015914917
Epoch 20, Loss: 1.0653512477874756
Epoch 30, Loss: 0.9900347590446472
Epoch 40, Loss: 0.9174520373344421
Epoch 50, Loss: 0.8495411276817322
Epoch 60, Loss: 0.7932561039924622
Epoch 70, Loss: 0.750663697719574
Epoch 80, Loss: 0.7206009030342102
Epoch 90, Loss: 0.6997930407524109
Epoch 100, Loss: 0.68441241979599
Epoch 10, Loss: 1.1716647148132324
Epoch 20, Loss: 1.1046124696731567
Epoch 30, Loss: 1.0511226654052734
Epoch 40, Loss: 1.0040850639343262
Epoch 50, Loss: 0.9563652873039246
Epoch 60, Loss: 0.9098278284072876
Epoch 70, Loss: 0.8677536845207214
Epoch 80, Loss: 0.831939160823822
Epoch 90, Loss: 0.8028867840766907
Epoch 100, Loss: 0.7781325578689575
Epoch 10, Loss: 1.1870919466018677
Epoch 20, Loss: 1.1594970226287842
Epoch 30, Loss: 1.1565465927124023
Epoch 40, Loss: 1.156393051147461
Epoch 50, Loss: 1.1563994884490967
Epoch 60, Loss: 1.1563352346420288
Epoch 70, Loss: 1.1563091278076172
Epoch 80, Loss: 1.1563079357147217
Epoch 90, Loss: 1.156306

C:\Users\Dieu\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10, 54])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim3, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, output_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define the dimensions of the input, hidden and output layers
input_dim = 54
hidden_dim1 = 32
hidden_dim2 = 16
hidden_dim3 = 8
output_dim = 1

# Create the stacked autoencoder model
model = AutoEncoder(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim)

# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):  # loop over the dataset multiple times
    optimizer.zero_grad()
    outputs = model(inputs)  # forward pass
    loss = criterion(outputs, inputs)  # calculate the loss
    loss.backward()  # backward pass
    optimizer.step()  # update the model parameters
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

NameError: name 'inputs' is not defined

In [9]:
# R = 

def detect_peaks(signal, fs, minimum_interval=0.1):
    """Détecte les pics dans le signal ECG en utilisant find_peaks."""
    min_distance = int(fs * minimum_interval)
    peaks, _ = find_peaks(signal, distance=min_distance)
    return peaks

def extract_windows(signal, peaks, window_size=54):
    """Extrait les fenêtres de 54 échantillons autour des pics détectés."""
    half_window = window_size // 2
    windows = []
    for peak in peaks:
        start = max(0, peak - half_window)
        end = min(len(signal), peak + half_window)
        window = signal[start:end]
        if len(window) < window_size:
            padding = window_size - len(window)
            if start == 0:
                window = np.pad(window, (padding, 0), mode='constant')
            else:
                window = np.pad(window, (0, padding), mode='constant')
        windows.append(window)
    return np.array(windows)

def normalize_windows(windows):
    """Normalise chaque fenêtre de 54 échantillons."""
    return (windows - np.mean(windows, axis=1, keepdims=True)) / np.std(windows, axis=1, keepdims=True)

def label_windows(peaks, qrs_indices, fs, min_distance=0.1):
    """Crée des étiquettes pour chaque fenêtre (1 = QRS, 0 = non-QRS)."""
    labels = np.zeros(len(peaks), dtype=int)
    qrs_set = set(qrs_indices)
    min_distance_samples = int(fs * min_distance)
    for i, peak in enumerate(peaks):
        if any(abs(peak - qrs) <= min_distance_samples for qrs in qrs_set):
            labels[i] = 1
    return labels

class Autoencoder(nn.Module):
    """Autoencodeur simple avec une couche cachée."""
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        layers = []
        for i in range(hidden_sizes):
            layers.append(nn.Linear(input_size if i == 0 else hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class QRSDetector(nn.Module):
    """Combinaison de l'autoencodeur et des MLP pour la détection de QRS."""
    def __init__(self, input_size=54, ae_hidden_size=27, mlp1_hidden=20, mlp2_hidden=4):
        super(QRSDetector, self).__init__()
        self.autoencoder = Autoencoder(input_size, ae_hidden_size)
        self.mlp1 = MLP(ae_hidden_size + 16, mlp1_hidden, 1)
        self.mlp2 = MLP(ae_hidden_size + 16, mlp2_hidden, 1)

    def forward(self, window, rr_intervals, amplitudes):
        reconstructed, encoded = self.autoencoder(window)
        features = torch.cat([encoded, rr_intervals, amplitudes], dim=1)
        output1 = self.mlp1(features)
        output2 = self.mlp2(features)
        return output1, output2

def autoencoder_loss(reconstructed, original):
    """Calcul de la perte MSE pour l'autoencodeur."""
    return F.mse_loss(reconstructed, original)

def fine_tuning_loss(output, targets):
    """Fonction de perte d'entropie croisée pour le fine-tuning."""
    return F.cross_entropy(output, targets)

In [10]:
def prepare_data(signal, qrs_indices, fs):
    """Prépare les données pour l'entraînement."""
    peaks = detect_peaks(signal, fs)
    windows = extract_windows(signal, peaks)
    windows = normalize_windows(windows)
    labels = label_windows(peaks, qrs_indices, fs)

    # Calcul des intervalles RR et des amplitudes des pics
    rr_intervals = np.diff(peaks, prepend=0)
    rr_intervals = np.concatenate([rr_intervals[-3:], rr_intervals])
    amplitudes = signal[peaks]

    # Préparation des caractéristiques pour chaque fenêtre
    features = []
    for i in range(len(peaks)):
        rr_window = rr_intervals[max(0, i-3):i+5]
        amp_window = amplitudes[max(0, i-3):i+5]
        if len(rr_window) < 8:
            rr_window = np.pad(rr_window, (8 - len(rr_window), 0), 'constant')
        if len(amp_window) < 8:
            amp_window = np.pad(amp_window, (8 - len(amp_window), 0), 'constant')
        features.append(np.concatenate([rr_window, amp_window]))
    features = np.array(features)

    return torch.tensor(windows, dtype=torch.float32), torch.tensor(features, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

def train_qrs_detector(signal, qrs_indices, fs=360, epochs=100, batch_size=16):
    """Entraîne le détecteur de QRS sur les 20% premiers du dataset."""
    # Préparation des données
    windows, features, labels = prepare_data(signal, qrs_indices, fs)

    # Utilisation des 20% premiers pour l'entraînement
    num_samples = int(len(windows) * 0.2)
    X_train, X_val = windows[:num_samples], windows[num_samples:]
    features_train, features_val = features[:num_samples], features[num_samples:]
    y_train, y_val = labels[:num_samples], labels[num_samples:]

    train_dataset = TensorDataset(X_train, features_train, y_train)
    val_dataset = TensorDataset(X_val, features_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialisation du modèle et des optimisateurs
    model = QRSDetector()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Boucle d'entraînement
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for windows_batch, features_batch, labels_batch in train_loader:
            optimizer.zero_grad()
            output1, output2 = model(windows_batch, features_batch[:, :8], features_batch[:, 8:])
            loss = custom_loss(output1, output2, labels_batch.view(-1, 1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for windows_batch, features_batch, labels_batch in val_loader:
                output1, output2 = model(windows_batch, features_batch[:, :8], features_batch[:, 8:])
                loss = custom_loss(output1, output2, labels_batch.view(-1, 1))
                val_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

    return model

def predict_qrs(signal, model, fs=360, threshold=0.5):
    """Prédit les complexes QRS dans le signal donné en utilisant le modèle entraîné."""
    peaks = detect_peaks(signal, fs)
    windows = extract_windows(signal, peaks)
    windows = normalize_windows(windows)

    rr_intervals = np.diff(peaks, prepend=0)
    rr_intervals = np.concatenate([rr_intervals[-3:], rr_intervals])
    amplitudes = signal[peaks]

    features = []
    for i in range(len(peaks)):
        rr_window = rr_intervals[max(0, i-3):i+5]
        amp_window = amplitudes[max(0, i-3):i+5]
        if len(rr_window) < 8:
            rr_window = np.pad(rr_window, (8 - len(rr_window), 0), 'constant')
        if len(amp_window) < 8:
            amp_window = np.pad(amp_window, (8 - len(amp_window), 0), 'constant')
        features.append(np.concatenate([rr_window, amp_window]))
    features = np.array(features)

    windows_tensor = torch.tensor(windows, dtype=torch.float32)
    features_tensor = torch.tensor(features, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        output1, output2 = model(windows_tensor, features_tensor[:, :8], features_tensor[:, 8:])
        predictions = (output2.squeeze() >= threshold).numpy()

    qrs_indices = peaks[predictions]
    return qrs_indices

In [11]:
qrs_indices = QRS
fs = 360

# Entraîner le détecteur de QRS
model = train_qrs_detector(ecg_signal, qrs_indices, fs)
predicted_qrs_indices = predict_qrs(ecg_signal, model, fs)

plt.figure(figsize=(12, 6))
plt.plot(ecg_signal, label='ECG Signal')
plt.scatter(predicted_qrs_indices, ecg_signal[predicted_qrs_indices], color='red', label='Predicted QRS')
plt.legend()
plt.show()

TypeError: 'int' object is not subscriptable

In [36]:
perf(QRS, predicted_qrs_indices, 36)

(1323, 1615, 150)

In [37]:
perf(QRS, predicted_qrs_indices, 36)

(1323, 1615, 150)

In [18]:
def perf(labels, peaks, minmax):
    x = np.concatenate([np.array(labels), np.array(peaks)]) #list(set(QRS + r_peaks))
    x.sort()
    diff = x[1:]-x[:-1]
    gps = np.concatenate([[0], np.cumsum(diff>=minmax)])
    temp = [x[gps==i] for i in range(gps[-1]+1)]
    TP = 0
    FP = 0
    FN = 0
    list_F = []
    for sublist in temp:
        if len(sublist) == 2:
            TP += 1
        else:
            list_F.append(sublist)
            if sublist[0] in labels:
                FN += 1
            else:
                FP += 1
    return TP, FP, FN #, list_F

def print_signal(signal, description):
    print(f"signal de longueur: {len(signal)}")
    plt.figure()
    plt.plot(signal)
    #plt.scatter(QRS, [signal[i] for i in QRS], color='red')
    plt.title(label= description)
    plt.show()
    
def print_signal_qrs(signal, description, qrs):
    print(f"signal de longueur: {len(signal)}")
    plt.figure()
    plt.plot(signal)
    plt.scatter(qrs, [signal[i] for i in qrs], color='red')
    plt.title(label= description)
    plt.show()